In [1]:
import os, sys
from PIL import Image
import random
import numpy as np
from scipy.misc import imsave

In [2]:
#apply transformations on a single image
def binarize_image(img_path, target_path, threshold=249):
    """Binarize an image."""
    image = Image.open(img_path)
    #image = image_file.convert('L')  # convert image to monochrome
    image = np.array(image)
   
    #print(image)
    #print(set(image.flatten()))
    image = binarize_array(image, threshold)
    #print(unique_count(image))
    imsave(target_path, image)
def unique_count(x):
    unique, counts = np.unique(x, return_counts=True)
    print (np.asarray((unique, counts)).T)
def binarize_array(numpy_array, threshold=0):
    """Binarize a numpy array."""
    for i in range(len(numpy_array)):
        for j in range(len(numpy_array[0])):
            if numpy_array[i][j] >= threshold:
                #label almost white pixels as class 1, and black as 0
                numpy_array[i][j] = 0.99
            else:
                numpy_array[i][j] = 0
    return numpy_array

In [5]:
#path = "/Users/deepitapai/Documents/TUM/MLMI Project/MontgomerySet_Original/"
path = "/Users/deepitapai/Documents/TUM/MLMIProject/One-Hundred-Layers-Tiramisu-master_1 copy/MontgomerySet_Original/"
if not os.path.exists(path+"/MergedMask_annotated_copy/"):
        os.makedirs(path+"/MergedMask_annotated_copy/")
        
for pic in os.listdir(path+"/MergedMask"):
        if pic.endswith(".png"): 
            binarize_image(path+"/MergedMask/"+pic,path+"/MergedMask_annotated_copy/"+pic)
           
    
   


/Users/deepitapai/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':


In [7]:
#flipping logic
def flip_image(image_path, saved_location):
    """
    Flip or mirror the image
 
    @param image_path: The path to the image to edit
    @param saved_location: Path to save the cropped image
    """
    image_obj = Image.open(image_path)
    rotated_image = image_obj.transpose(Image.FLIP_LEFT_RIGHT)
    rotated_image.save(saved_location)
    #rotated_image.show()

def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col= image.shape
        mean = 0
        var = 1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col))
        print(gauss)
        gauss = gauss.reshape(row,col)
        noisy = image + gauss
        #print(set(gauss.flatten()))
        return noisy
    elif noise_typ == "s&p":
        #row,col,ch = image.shape
        row,col = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1 - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy


In [15]:
path = "/Users/deepitapai/Documents/TUM/MLMI Project/MontgomerySet_Original/"
for item in ["MergedMask","ResizedCXR"]:
    if not os.path.exists(path+"/"+item+"_flipped/"):
            os.makedirs(path+"/"+item+"_flipped/")

    for pic in os.listdir(path+"/"+item):
            if pic.endswith(".png"): 
                flip_image(path+"/"+item+"/"+pic,path+"/"+item+"_flipped"+"/"+"flipped_"+pic)
#flipping logic

In [16]:
# add salt and pepper noise to original images, but also create unaltered copies of masks
path = "/Users/deepitapai/Documents/TUM/MLMI Project/MontgomerySet_Original/"
for item in ["MergedMask","ResizedCXR"]:
    if not os.path.exists(path+"/"+item+"_saltPepper/"):
            os.makedirs(path+"/"+item+"_saltPepper/")

    for pic in os.listdir(path+"/"+item):
            if pic.endswith(".png"): 
                image_path = path+"/"+item+"/"+pic
                save_path = path+"/"+item+"_saltPepper"+"/"+"saltPepper_"+pic
                image = Image.open(image_path)
                if item is not "MergedMask":
                    image = np.array(image)
                    image = noisy(noise_typ = "s&p", image = image)
                    imsave(save_path, image)
                else:
                    image.save(save_path)
                    
#flipping logic

/Users/deepitapai/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  from ipykernel import kernelapp as app


In [17]:
# add salt and pepper noise to original images, but also create unaltered copies of masks
path = "/Users/deepitapai/Documents/TUM/MLMI Project/MontgomerySet_Original/"
for item in ["MergedMask","ResizedCXR"]:
    if not os.path.exists(path+"/"+item+"_gaussianNoise/"):
            os.makedirs(path+"/"+item+"_gaussianNoise/")

    for pic in os.listdir(path+"/"+item):
            if pic.endswith(".png"): 
                image_path = path+"/"+item+"/"+pic
                save_path = path+"/"+item+"_gaussianNoise"+"/"+"gaussianNoise_"+pic
                image = Image.open(image_path)
                if item is not "MergedMask":
                    image = np.array(image)
                    image = noisy(noise_typ = "gauss", image = image)
                    imsave(save_path, image)
                else:
                    image.save(save_path)
                    
#flipping logic



[[-0.17280021  0.9017595   0.62153575 ... -0.63755517  1.01374612
  -1.56318543]
 [-0.98182742  0.7613173   1.44452461 ... -0.60444528 -0.26310367
  -0.97274428]
 [-0.23602182  1.56360143 -1.92696951 ...  1.24853682 -0.61135696
  -1.95447803]
 ...
 [-0.01151739  0.446223   -0.69942136 ...  0.65338474 -0.19708859
  -0.24975312]
 [ 1.56396157 -0.32754956 -0.57259429 ...  0.30143359 -0.08752206
   0.68220367]
 [-0.86265507 -1.26540075 -0.59156811 ...  0.74942598  0.32672757
   0.04187388]]
[[-1.02862179  0.74757159  0.56783316 ... -2.0275706  -1.96810743
   0.96747766]
 [-0.45477397  0.61356514  0.53220994 ...  0.27269034  0.59869382
   0.02614254]
 [ 1.9396696  -0.1729767   0.44539118 ...  0.10838415  0.57404646
  -0.12389914]
 ...
 [-0.1316647   0.17275999  1.67217829 ... -0.15514373  0.20509399
   1.1840832 ]
 [-1.06248402  0.3416679   1.03238132 ...  0.03594128  0.32948346
   0.54435342]
 [ 0.99440862  1.06689831  0.98909159 ... -1.20034573 -0.28389161
  -1.4756372 ]]
[[ 0.3524059  -0

/Users/deepitapai/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  from ipykernel import kernelapp as app


[[ 1.56875043 -1.74065491  0.21940373 ... -0.34134956  1.30619447
  -0.53112904]
 [-0.95872483  0.35085581  0.79439328 ...  1.92499606 -1.20254209
  -0.07943058]
 [-0.18920196  0.34177881 -0.38725675 ...  0.01397216 -1.86864611
   0.91547268]
 ...
 [-1.87290005 -0.41964173 -0.9643409  ...  0.76359515 -0.26652992
   1.46598455]
 [ 1.08008056 -0.55921433 -0.86888517 ...  0.80296481  0.70983945
  -0.77851088]
 [-1.87909484  0.36532136 -1.66679649 ... -0.5711332   0.99772008
   1.55227412]]
[[ 2.30117654  0.55509946  1.48883461 ... -0.41342682 -0.80658233
  -1.10925175]
 [ 1.05610471 -0.84932551 -1.09560421 ...  1.20830265 -1.02340273
  -1.11670476]
 [-0.60849728  0.86889116  0.60016603 ... -0.07481262  0.52146128
  -0.13717418]
 ...
 [ 0.93038415  0.93279925 -0.5641121  ...  0.45912051 -0.17148784
   0.88814266]
 [-1.58750242 -0.0378601   2.07584114 ... -1.34495208  1.31875338
   0.61933905]
 [ 0.37756891 -1.349069    0.96793336 ...  1.30259461  1.8091646
  -0.69950849]]
[[-1.43928448 -0.

[[-1.88346227  1.80835545  0.36691721 ...  0.50556813 -0.08624992
  -0.42166247]
 [-1.25282415 -0.71182219  0.07472544 ...  0.28273406 -0.95351065
   0.42378744]
 [-0.55331645  0.89363429  0.44807943 ... -0.50777999 -0.07186861
  -0.81198838]
 ...
 [-0.92277486 -0.6020731   1.90742309 ... -0.85893056  1.66174819
   0.14677238]
 [ 2.55001317  0.28045439 -1.73925039 ...  1.07914455  0.95419483
   1.39632716]
 [-0.13919943 -1.50528644 -0.15936126 ... -0.17436823  1.38810801
  -0.09259108]]
[[ 1.06816468  0.55887706 -1.67942387 ... -0.30064512  1.12138084
  -0.4026791 ]
 [ 0.74318154  0.64479784  0.15508266 ...  0.62144618  1.29961194
  -1.42471431]
 [ 0.2671093   1.40237248 -0.98775623 ... -0.89697855  1.23706932
  -0.67246457]
 ...
 [-0.51738873 -0.75472655  0.56276902 ... -0.93667089  0.10734951
  -0.0358289 ]
 [-0.1092859   0.28976896 -0.64334783 ... -2.12524783 -1.25033009
   0.20889986]
 [-0.86387022  0.97436593 -0.47775732 ...  1.98544848  0.09326597
   0.10923633]]
[[ 0.72330461 -0

[[-0.56975753 -0.35331627 -1.8812739  ... -0.25255887  0.04141567
  -2.05922003]
 [-0.47098137  0.83003071  1.20561975 ...  0.77624171 -0.04138074
   0.33918001]
 [ 2.04084841 -0.68825402  1.26840903 ...  0.76308002 -0.02968611
  -0.95478809]
 ...
 [ 0.71980101 -0.34660229  0.81256488 ...  0.2165882   0.62403621
   0.14166942]
 [ 0.79964905 -1.16630446  0.45338838 ...  1.5872486  -0.08842097
   0.18039828]
 [-0.62171429  1.3865212   1.54814728 ...  0.58807973 -0.12586084
  -2.1382328 ]]
[[ 0.3393425   0.15550246 -0.40060699 ... -0.03510246 -0.1020563
   2.18829085]
 [ 0.98685447 -0.87061856 -1.11818106 ...  0.75158832 -0.12326129
  -0.79224897]
 [ 0.5922304   0.10714683 -0.12513229 ...  1.43305904 -1.54019962
   0.05168632]
 ...
 [-0.15390543  0.26562115 -0.41216736 ... -0.01599024  1.3353172
   1.64161978]
 [-0.88844685  0.52169118  1.08501196 ...  0.35721211 -0.90457039
   0.33932237]
 [ 1.09335029 -0.37667702 -0.14630377 ...  1.43805698 -0.21300196
   0.36273656]]
[[-0.6192038   0.3

[[-0.40004747  0.46823462 -0.95590632 ... -0.71541168 -0.17369899
   0.31747386]
 [ 0.70542767 -1.93886301 -0.08356079 ...  2.55054272 -0.88842079
  -1.25991257]
 [ 1.9253435   0.24353969  1.18614448 ...  0.28972621 -0.16479194
   0.99778967]
 ...
 [-2.92770977 -0.93325357 -1.77418494 ...  0.08512846  0.87380533
  -2.13259495]
 [ 2.29894381  0.8023591   0.26584773 ...  0.08425831  0.35656276
   0.94600144]
 [ 0.22001271  0.71175736 -0.73277738 ...  1.09479682 -0.3650882
  -1.49940556]]
[[-0.15737974 -1.0477258  -0.09917091 ...  0.37254922  0.50204236
  -2.14058779]
 [ 0.19018268 -1.15008809 -0.14067481 ... -0.13895046  0.38957793
  -0.26407093]
 [-0.50382694 -0.10407313  0.7495507  ...  0.70609702  0.28068492
   0.53744494]
 ...
 [-1.63065971 -0.08381851  0.57726232 ...  0.60349275  0.43124375
  -0.7622378 ]
 [ 0.15190383  1.30225543 -0.59465395 ... -0.56348527 -0.34497552
   1.62195376]
 [ 0.87186614  0.22636054 -0.64916331 ...  0.35271309 -0.73887092
   1.23610901]]
[[ 0.69864238 -0.

[[ 1.0589395  -0.67114277  0.46720549 ...  0.59677066  1.75757122
  -1.94130131]
 [-0.7176841   2.48100207  1.8508352  ...  0.73250179  0.41880932
   1.47912271]
 [-0.33634304  0.30797187 -1.04182975 ... -0.01480213  0.93132748
   1.56527423]
 ...
 [ 0.57452208  0.92601432  0.29322883 ...  1.29986891 -0.23239026
   0.42001288]
 [ 0.02164799  0.89284691 -0.02345976 ...  1.28004757 -0.15662103
   0.59897162]
 [-0.52865705  0.35482443  0.17904595 ...  1.19909164 -1.59293125
   0.98240182]]
[[ 1.38921082  0.41540537 -0.88782954 ...  0.60686606 -0.3280484
   0.00373971]
 [ 1.09154456  1.10271102  1.01281202 ...  0.58962134  0.61017659
  -2.03359106]
 [-1.45573891 -0.54280059  0.86705712 ... -1.64873485  1.16828489
  -0.11906268]
 ...
 [ 0.42293298 -0.22263554 -0.52684045 ...  2.0301465  -1.02058854
  -1.89618274]
 [-1.36928735  0.42423848  1.16674509 ... -1.33567322  0.71771852
  -1.64881686]
 [ 0.09658332 -0.62404514 -0.13451283 ...  1.2790117  -0.63456866
   1.86985562]]
[[-0.1611399   0.

[[ 0.77664385  1.91340475  0.8874358  ...  0.50158412 -0.45315971
  -0.63944539]
 [ 0.10812598 -1.73105669 -0.72887301 ...  1.3517404   0.72184666
  -0.18876994]
 [ 0.18092428  0.34263788  0.18107327 ...  0.59971024  0.84860495
  -1.54530035]
 ...
 [ 0.77767672  0.9701324  -0.31973031 ...  0.95715971 -2.19165943
   2.28290286]
 [-0.96838713  1.46776042 -0.20329034 ...  2.30887277  1.60799217
   1.06846261]
 [ 0.44179432 -1.96724162 -0.40171574 ... -0.74066754  1.8713167
  -0.58944089]]
[[ 1.16068652  0.38509465  0.50487722 ...  0.7674904   0.89034232
   1.65157274]
 [-0.68704398 -0.17659301  0.40372536 ...  0.26532206  0.13158214
  -0.9483192 ]
 [ 0.39121281 -0.67332613 -1.45898213 ... -0.3224677   1.70123976
   1.02974645]
 ...
 [ 1.34620414 -0.82458024 -0.12239642 ...  0.62171656  0.18292932
  -0.21892375]
 [ 0.3728357  -0.35420784  0.17017453 ...  0.46720353 -0.71570648
   1.27059478]
 [-0.02854242 -1.84538676  1.29554302 ...  0.41694821 -0.30782588
   0.03272766]]
[[-2.31964687 -1.

In [78]:
import Augmentor

In [84]:
p = Augmentor.Pipeline("/Users/deepitapai/Documents/TUM/MLMI Project/MontgomerySet_Original/ResizedCXR")
# Point to a directory containing ground truth data.
# Images with the same file names will be added as ground truth data
# and augmented in parallel to the original data.
p.ground_truth("/Users/q453991/Downloads/Archiv copy/MergedMask")

Initialised with 138 image(s) found.
Output directory set to /Users/q453991/Downloads/Archiv copy/ResizedCXR/output.138 ground truth image(s) found.


In [85]:

p.random_contrast(probability=1,min_factor= .05, max_factor=.1)
p.process()

Processing <PIL.Image.Image image mode=L size=480x360 at 0x1102704E0>: 100%|██████████| 138/138 [00:01<00:00, 126.04 Samples/s]
